<a href="https://colab.research.google.com/github/doyeo-n/-Capstone-8_code/blob/main/%5B2%5D%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#필요한 패키지 설치
!pip install fancyimpute
!pip install missingpy
!pip install impyute
!pip install sklearn
!pip install missingpy
!pip install impyute
!pip install sklearn
!pip install ujson
!pip install utils
!pip install ipdb
!pip install shap
!pip install pykalman
!pip install ujson
!pip install utils
!pip install ipdb
!pip install shap
!pip install pykalman

#torch
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import shap

import numpy as np
from numpy import ma
import pandas as pd
import pickle as pickle
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import ujson as json
import math
import os
import re
import time
import ipdb
from ipdb import set_trace
import scipy
import scipy.stats
from pykalman import KalmanFilter

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import seaborn as sb
# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from impyute.imputation.cs import mice
import shap
import numpy as np
from numpy import ma
import pandas as pd
import pickle as pickle
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import ujson as json
import math
import os
import re
import time
import ipdb
from ipdb import set_trace
import scipy
import scipy.stats
from pykalman import KalmanFilter
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

#데이터 불러오기 및 전처리
def load_data(path, input_list, index_col, output_col):
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    original = pd.read_csv(path, encoding = "cp949", thousands = ',') 
    original = original[input_list]
    original[index_col] = pd.to_datetime(original[index_col])
    original = original.set_index(index_col)
    output_var = [output_col]
    attributes = input_list
    #로그 스케일 변환
    for attr in attributes[2:-1]:
      print(attr)
      original[attr] = np.log1p(original[attr])
    data = original

    #EDA
    fig = plt.figure(figsize = (20, 10)) 
    rows = 3; cols = 5
    o = 1
    for attr in attributes[1:]:
        ax = fig.add_subplot(rows, cols, o)
        im= ax.hist(data[attr], bins = 20)
        ax.set_title(attr)
        o += 1
    plt.show()
    return data, original

def kalman(eutrophication):
  data_kalman = eutrophication.copy()
  for i in range(1, len(data_kalman.columns)):
    kf = KalmanFilter(initial_state_mean = np.nanmean(data_kalman.iloc[:,i]), n_dim_obs = 1)
    X_masked = pd.isna(data_kalman.iloc[:,i])
    X =data_kalman.iloc[:, i]
    X = ma.array(X)
    X.mask = X_masked
    (smoothed_state_means, smoothed_state_covariances) = kf.em(X, n_iter=5).smooth(X)
    data_kalman.iloc[:,i] = np.where(pd.isna(data_kalman.iloc[:, i]) == True, smoothe_state_means.reshape(-1), dta_kalman.iloc[:, i])
  return data_kalman



def imputation(imput_method):
  !pip install impyute
  !pip install sklearn
  !pip install fancyimpute
  from pandas import read_csv
  from matplotlib import pyplot
  from impyute.imputation.cs import mice
  import pandas as pd
  import numpy as np
  import datetime
  from datetime import datetime

  """
  start_date = data.index[0]
  start_date = datetime.now().strftime('%Y-%m-%d')
  #start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
  end_date = data.index[-1]
  end_date = datetime.now().strftime('%Y-%m-%d')
  #end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
  """
  #print(f' start date : {start_date}, end date : {end_date}')
  data, original = load_data(path, input_list, index_col, output_col)
  eutrophication = pd.DataFrame(np.array(pd.date_range(start="2012-08-27",end="2021-04-26")))
  eutrophication.columns = [index_col]
  eutrophication[index_col] = pd.to_datetime(eutrophication[index_col])
  eutrophication = pd.merge(left = eutrophication, right = original, on = 'date', how='outer')
  for i in range(1, len(eutrophication.columns)):
    name_c = eutrophication.columns[i]
    eutrophication[name_c] = np.where(eutrophication[name_c]>=0, eutrophication[name_c], np.nan)
  print(eutrophication)
  if imput_method == 'kalman':
    imputed_data = kalman(eutrophication)
  elif imput_method == 'mice':
    data = eutrophication
    data_mice = data.copy()
    data_mice = mice((data_mice.iloc[:, 1:].values))
    data_mice= pd.DataFrame(data_mice)
    print(data_mice)
    data_mice.insert(0, 'date', data[index_col])    
    data_mice.columns = data.columns
    print(data_mice.columns)
    imputed_data = data_mice

  groups = [0, 1, 2, 3, 5, 6, 7,8,9]
  i = 1
  # plot each column
  pyplot.figure(figsize = (20,15))
  for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(imputed_data.iloc[:, group])
    pyplot.title(imputed_data.columns[group], y=0.5, loc='right')
    i += 1
  pyplot.show()
  return eutrophication, imputed_data

In [ ]:

#데이터 불러오기 및 전처리
def load_data(path, input_list, index_col, output_col):
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    original = pd.read_csv(path, encoding = "cp949", thousands = ',') 
    original = original[input_list]
    original[index_col] = pd.to_datetime(original[index_col])
    original = original.set_index(index_col)
    output_var = [output_col]
    attributes = input_list
    #로그 스케일 변환
    for attr in attributes[2:-1]:
      print(attr)
      original[attr] = np.log1p(original[attr])
    data = original

    #EDA
    fig = plt.figure(figsize = (20, 10)) 
    rows = 3; cols = 5
    o = 1
    for attr in attributes[1:]:
        ax = fig.add_subplot(rows, cols, o)
        im= ax.hist(data[attr], bins = 20)
        ax.set_title(attr)
        o += 1
    plt.show()
    return data, original

def imputation(data, original):
  !pip install impyute
  !pip install sklearn
  !pip install fancyimpute
  from pandas import read_csv
  from matplotlib import pyplot
  from impyute.imputation.cs import mice
  import pandas as pd
  import numpy as np
  import datetime
  from datetime import datetime

  """
  start_date = data.index[0]
  start_date = datetime.now().strftime('%Y-%m-%d')
  #start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
  end_date = data.index[-1]
  end_date = datetime.now().strftime('%Y-%m-%d')
  #end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
  """
  #print(f' start date : {start_date}, end date : {end_date}')
  data, original = load_data(path, input_list, index_col, output_col)
  eutrophication = pd.DataFrame(np.array(pd.date_range(start="2012-08-27",end="2021-04-26")))
  eutrophication.columns = [index_col]
  eutrophication[index_col] = pd.to_datetime(eutrophication[index_col])
  eutrophication = pd.merge(left = eutrophication, right = original, on = 'date', how='outer')

  for i in range(1, len(eutrophication.columns)):
    name_c = eutrophication.columns[i]
    eutrophication[name_c] = np.where(eutrophication[name_c]>=0, eutrophication[name_c], np.nan)
  print(eutrophication)
  return eutrophication

def rea_imp(eutrophication, imput_method):
  if imput_method == 'mice':
    data = eutrophication
    data_mice = data.copy()
    data_mice = mice((data_mice.iloc[:, 1:].values))
    data_mice= pd.DataFrame(data_mice)
    print(data_mice)
    data_mice.insert(0, 'date', data[index_col])    
    data_mice.columns = data.columns
    print(data_mice.columns)
    imputed_data = data_mice
  groups = [0, 1, 2, 3, 5, 6, 7,8,9]
  i = 1
  # plot each column
  pyplot.figure(figsize = (20,15))
  for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(imputed_data.iloc[:, group])
    pyplot.title(imputed_data.columns[group], y=0.5, loc='right')
    i += 1
  pyplot.show()
  return imputed_data

In [ ]:
#결정사항
input_list = ['date', 'temp_WQ', 'irr', 'RT', 'prec_2', 'DO_WQ', 'BOD_WQ', 'TN_WQ', 'TP_WQ', 'EColi_WQ', 'log_cyan']
output_col = 'log_cyan'
index_col = 'date'
path = './gdrive/My Drive/Colab Notebooks/chang.csv'
n_input = 7
#mice 또는 kalman 입력
imput_method = 'kalman' 


#데이터 전처리
original, data = load_data(path,input_list, index_col, output_col)
eutrophication = imputation(original, data)
data_kalman = data.copy()
for i in range(1, len(data_kalman.columns)-1):
 kf = KalmanFilter(initial_state_mean=np.nanmean(data_kalman.iloc[:, i]), n_dim_obs=1)
 X_masked = pd.isna(data_kalman.iloc[:, i])
 X = data_kalman.iloc[:, i]
 X = ma.array(X)
 X.mask = X_masked
 (smoothed_state_means, smoothed_state_covariances) = kf.em(X, n_iter=5).smooth(X)
 data_kalman.iloc[:, i] = np.where(pd.isna(data_kalman.iloc[:, i]) == True, smoothed_state_means.reshape(-1), data_kalman.iloc[:, i])
imputed_data = data_kalman

if imput_method == 'mice':
  imputed_data = rea_imp(eutrophication, imput_method)